In [2]:
import json
import requests
import pyotp
import hashlib
from urllib import parse
import sys

# This script will only work if TOTP is enabled. 
# You can enable TOTP using this link: https://myaccount.fyers.in/ManageAccount >> External 2FA TOTP >> click on "Enable".

# Client Information (ENTER YOUR OWN INFO HERE! Data varies from users and app types)
CLIENT_ID = "XA00330"       # Your Fyers Client ID
PIN = "4445"                # User pin for Fyers account
APP_ID = "8I122G8NSD"        # App ID from MyAPI dashboard (https://myapi.fyers.in/dashboard). The format is appId-appType. 
# Example: YIXXXXXSE-100. In this code, YIXXXXXSE is the APP_ID and 100 is the APP_TYPE
APP_TYPE = "100"
APP_SECRET = "E3WTRCMOCL"   # App Secret from myapi dashboard (https://myapi.fyers.in/dashboard)
TOTP_SECRET_KEY = "DV42ZDKZPMX6U7TH7272FVMYY4OQTINQ"  # TOTP secret key, copy the secret while enabling TOTP.

REDIRECT_URI = "https://www.google.com"  # Redirect URL from the app

# NOTE: Do not share these secrets with anyone.


# API endpoints
BASE_URL = "https://api-t2.fyers.in/vagator/v2"
BASE_URL_2 = "https://api-t1.fyers.in/api/v3"
URL_VERIFY_CLIENT_ID = BASE_URL + "/send_login_otp"
URL_VERIFY_TOTP = BASE_URL + "/verify_otp"
URL_VERIFY_PIN = BASE_URL + "/verify_pin"
URL_TOKEN = BASE_URL_2 + "/token"
URL_VALIDATE_AUTH_CODE = BASE_URL_2 + "/validate-authcode"

SUCCESS = 1
ERROR = -1

def verify_client_id(client_id):
    try:
        payload = {
            "fy_id": client_id,
            "app_id": "2"
        }

        result_string = requests.post(url=URL_VERIFY_CLIENT_ID, json=payload)
        # print("result_string : ", result_string.text)
        if result_string.status_code != 200:
            return [ERROR, result_string.text]

        result = json.loads(result_string.text)
        request_key = result["request_key"]

        return [SUCCESS, request_key]
    
    except Exception as e:
        return [ERROR, e]
    

def generate_totp(secret):
    try:
        generated_totp = pyotp.TOTP(secret).now()
        return [SUCCESS, generated_totp]
    
    except Exception as e:
        return [ERROR, e]


def verify_totp(request_key, totp):
    try:
        payload = {
            "request_key": request_key,
            "otp": totp
        }

        result_string = requests.post(url=URL_VERIFY_TOTP, json=payload)
        if result_string.status_code != 200:
            return [ERROR, result_string.text]

        result = json.loads(result_string.text)
        request_key = result["request_key"]

        return [SUCCESS, request_key]
    
    except Exception as e:
        return [ERROR, e]


def verify_PIN(request_key, pin):
    try:
        payload = {
            "request_key": request_key,
            "identity_type": "pin",
            "identifier": pin
        }

        result_string = requests.post(url=URL_VERIFY_PIN, json=payload)
        if result_string.status_code != 200:
            return [ERROR, result_string.text]
    
        result = json.loads(result_string.text)
        access_token = result["data"]["access_token"]

        return [SUCCESS, access_token]
    
    except Exception as e:
        return [ERROR, e]


def token(client_id, app_id, redirect_uri, app_type, access_token):
    try:
        payload = {
            "fyers_id": client_id,
            "app_id": app_id,
            "redirect_uri": redirect_uri,
            "appType": app_type,
            "code_challenge": "",
            "state": "sample_state",
            "scope": "",
            "nonce": "",
            "response_type": "code",
            "create_cookie": True
        }
        headers={'Authorization': f'Bearer {access_token}'}

        result_string = requests.post(
            url=URL_TOKEN, json=payload, headers=headers
        )

        if result_string.status_code != 308:
            return [ERROR, result_string.text]

        result = json.loads(result_string.text)
        url = result["Url"]
        auth_code = parse.parse_qs(parse.urlparse(url).query)['auth_code'][0]

        return [SUCCESS, auth_code]
    
    except Exception as e:
        return [ERROR, e]


def sha256_hash(appId, appType, appSecret):
    message = f"{appId}-{appType}:{appSecret}"
    message = message.encode()
    sha256_hash = hashlib.sha256(message).hexdigest()
    return sha256_hash


def validate_authcode(auth_code):
    try:
        app_id_hash = sha256_hash(appId=APP_ID, appType=APP_TYPE, appSecret=APP_SECRET)
        payload = {
            "grant_type": "authorization_code",
            "appIdHash": app_id_hash,
            "code": auth_code,
        }

        result_string = requests.post(url=URL_VALIDATE_AUTH_CODE, json=payload)
        if result_string.status_code != 200:
            return [ERROR, result_string.text]

        result = json.loads(result_string.text)
        access_token = result["access_token"]

        return [SUCCESS, access_token]
    
    except Exception as e:
        return [ERROR, e]


def main():
    # Step 1 - Retrieve request_key from verify_client_id Function
    verify_client_id_result = verify_client_id(client_id=CLIENT_ID)
    if verify_client_id_result[0] != SUCCESS:
        print(f"verify_client_id failure - {verify_client_id_result[1]}")
        sys.exit()
    else:
        print("verify_client_id success")

    # Step 2 - Generate totp
    generate_totp_result = generate_totp(secret=TOTP_SECRET_KEY)
    if generate_totp_result[0] != SUCCESS:
        print(f"generate_totp failure - {generate_totp_result[1]}")
        sys.exit()
    else:
        print("generate_totp success")

    # Step 3 - Verify totp and get request key from verify_totp Function.
    request_key = verify_client_id_result[1]
    totp = generate_totp_result[1]
    verify_totp_result = verify_totp(request_key=request_key, totp=totp)
    if verify_totp_result[0] != SUCCESS:
        print(f"verify_totp_result failure - {verify_totp_result[1]}")
        sys.exit()
    else:
        print("verify_totp_result success")
    
    # Step 4 - Verify pin and send back access token
    request_key_2 = verify_totp_result[1]
    verify_pin_result = verify_PIN(request_key=request_key_2, pin=PIN)
    if verify_pin_result[0] != SUCCESS:
        print(f"verify_pin_result failure - {verify_pin_result[1]}")
        sys.exit()
    else:
        print("verify_pin_result success")
    
    # Step 5 - Get auth code for API V3 App from trade access token
    token_result = token(
        client_id=CLIENT_ID, app_id=APP_ID, redirect_uri=REDIRECT_URI, app_type=APP_TYPE,
        access_token=verify_pin_result[1]
    )
    if token_result[0] != SUCCESS:
        print(f"token_result failure - {token_result[1]}")
        sys.exit()
    else:
        print("token_result success")

    # Step 6 - Get API V3 access token from validating auth code
    auth_code = token_result[1]
    validate_authcode_result = validate_authcode(auth_code=auth_code)
    if token_result[0] != SUCCESS:
        print(f"validate_authcode failure - {validate_authcode_result[1]}")
        sys.exit()
    else:
        print("validate_authcode success")
    
    access_token = APP_ID + "-" + APP_TYPE + ":" + validate_authcode_result[1]

    print(f"\naccess_token - {access_token}\n")

if __name__ == "__main__":
    main()


verify_client_id success
generate_totp success
verify_totp_result success
verify_pin_result success
token_result success
validate_authcode success

access_token - 8I122G8NSD-100:eyJ0eXAiOiJKV1QiLCJhbGciOiJIUzI1NiJ9.eyJpc3MiOiJhcGkuZnllcnMuaW4iLCJpYXQiOjE3MzgxMzc0NTEsImV4cCI6MTczODE5NzAzMSwibmJmIjoxNzM4MTM3NDUxLCJhdWQiOlsieDowIiwieDoxIiwieDoyIiwiZDoxIiwiZDoyIiwieDoxIiwieDowIl0sInN1YiI6ImFjY2Vzc190b2tlbiIsImF0X2hhc2giOiJnQUFBQUFCbm1kOXI4Q0lOc3c2MnNQVjIzRVZpYzB3SFEzYkg3R2xHVVdpZWJRS0pSV3Y2SUl0N1FjSDlnWFlyeEtJaTczNllIQ3YtdDI4TVY3ZzFzYVlkamp5emludEItdEw1cTlBZ2ZUMWRlM1M3S1BLbWVIaz0iLCJkaXNwbGF5X25hbWUiOiJBU0hJU0ggU0hBUk1BIiwib21zIjoiSzEiLCJoc21fa2V5IjoiMTczZjkwNzg0YTY4YjJlYTkxNDg5YTY2ODU1NjBiMTYwNWVhZjhiYmRkNDg5Y2FjMTU3NTk3ZTgiLCJpc0RkcGlFbmFibGVkIjoiTiIsImlzTXRmRW5hYmxlZCI6Ik4iLCJmeV9pZCI6IlhBMDAzMzAiLCJhcHBUeXBlIjoxMDAsInBvYV9mbGFnIjoiTiJ9.0NaPdJYqPgDUVHs0Z8F2vOCMzG9_6D4VHXmcrdFUXLY



In [ ]:
# to get client data in response, for verification of API

client_id = "XC4XXXXM-100"
access_token = "eyJ0eXXXXXXXX2c5-Y3RgS8wR14g"

# Initialize the FyersModel instance with your client_id, access_token, and enable async mode
fyers = fyersModel.FyersModel(client_id=client_id, is_async=False, token=access_token, log_path="")

# Make a request to get the user profile information
response = fyers.get_profile()

# Print the response received from the Fyers API
print(response)

#below is the sample response you will recieve from fyers
----------------------------------------------------------------------------------------------------------------
Sample Response
----------------------------------------------------------------------------------------------------------------
  {
    "s":"ok",
    "code":200,
    "message":"",
    "data":{
        "fy_id":"XXXXX86",
        "name":"VINAY",
        "image":"https://fyers-user-details.s3.amazonaws.com/image/FK6107548224?X-Amz-Algorithm=AWS4-HMAC",
        "display_name":"VKM",
        "pin_change_date":"15-12-2022 09:24:05",
        "email_id":"xxxnayxm@gmxxl.com",
        "pwd_change_date":"None",
        "PAN":"XXXXXXXXXX",
        "mobile_number":"9XXXXX678",
        "totp":true,
        "pwd_to_expire":90
    }
  }


In [ ]:
# to fetch the market data from API Option Chain

from fyers_apiv3 import fyersModel
client_id = "YYYYYYY-100"
access_token = "XXXXXXXXXXXXX"
# Initialize the FyersModel instance with your client_id, access_token, and enable async mode
fyers = fyersModel.FyersModel(client_id=client_id, token=access_token,is_async=False, log_path="")
data = {
    "symbol":"NSE:TCS-EQ",
    "strikecount":1,
    "timestamp": ""
}
response = fyers.optionchain(data=data);
print(response)
 ------------------------------------------------------------------------------------------------------------------------------------------
 Sample Success Response 
 ------------------------------------------------------------------------------------------------------------------------------------------
 {
  "code": 200,
  "data": {
    "callOi": 10038175,
    "expiryData": [
      {
        "date": "25-04-2024",
        "expiry": "1714039200"
      },
      {
        "date": "30-05-2024",
        "expiry": "1717063200"
      },
      {
        "date": "27-06-2024",
        "expiry": "1719482400"
      }
    ],
    "indiavixData": {
      "ask": 0,
      "bid": 0,
      "description": "INDIAVIX-INDEX",
      "ex_symbol": "INDIAVIX",
      "exchange": "NSE",
      "fyToken": "101000000026017",
      "ltp": 10.55,
      "ltpch": -2.15,
      "ltpchp": -16.93,
      "option_type": "",
      "strike_price": -1,
      "symbol": "NSE:INDIAVIX-INDEX"
    },
    "optionsChain": [
      {
        "ask": 3880.15,
        "bid": 3880.05,
        "description": "TATA CONSULTANCY SERV LT",
        "ex_symbol": "TCS",
        "exchange": "NSE",
        "fp": 3876.65,
        "fpch": 14.2,
        "fpchp": 0.37,
        "fyToken": "101000000011536",
        "ltp": 3880.15,
        "ltpch": 15.55,
        "ltpchp": 0.4,
        "option_type": "",
        "strike_price": -1,
        "symbol": "NSE:TCS-EQ"
      },
      {
        "ask": 34.9,
        "bid": 34.35,
        "fyToken": "1011240425139431",
        "ltp": 34.8,
        "ltpch": 2.7,
        "ltpchp": 8.41,
        "oi": 99575,
        "oich": -3325,
        "oichp": -3.23,
        "option_type": "CE",
        "prev_oi": 102900,
        "strike_price": 3860,
        "symbol": "NSE:TCS24APR3860CE",
        "volume": 202650
      },
      {
        "ask": 19.3,
        "bid": 19,
        "fyToken": "1011240425139432",
        "ltp": 19.05,
        "ltpch": -12.4,
        "ltpchp": -39.43,
        "oi": 159075,
        "oich": 28525,
        "oichp": 21.85,
        "option_type": "PE",
        "prev_oi": 130550,
        "strike_price": 3860,
        "symbol": "NSE:TCS24APR3860PE",
        "volume": 304150
      },
      {
        "ask": 24.85,
        "bid": 24.55,
        "fyToken": "1011240425133432",
        "ltp": 24.95,
        "ltpch": -0.55,
        "ltpchp": -2.16,
        "oi": 165900,
        "oich": 9975,
        "oichp": 6.4,
        "option_type": "CE",
        "prev_oi": 155925,
        "strike_price": 3880,
        "symbol": "NSE:TCS24APR3880CE",
        "volume": 543025
      },
      {
        "ask": 29.35,
        "bid": 28.8,
        "fyToken": "1011240425133433",
        "ltp": 29.2,
        "ltpch": -14.1,
        "ltpchp": -32.56,
        "oi": 98175,
        "oich": 28350,
        "oichp": 40.6,
        "option_type": "PE",
        "prev_oi": 69825,
        "strike_price": 3880,
        "symbol": "NSE:TCS24APR3880PE",
        "volume": 199500
      },
      {
        "ask": 17.8,
        "bid": 17.6,
        "fyToken": "1011240425139433",
        "ltp": 17.75,
        "ltpch": -1.4,
        "ltpchp": -7.31,
        "oi": 631050,
        "oich": 23275,
        "oichp": 3.83,
        "option_type": "CE",
        "prev_oi": 607775,
        "strike_price": 3900,
        "symbol": "NSE:TCS24APR3900CE",
        "volume": 981925
      },
      {
        "ask": 42.45,
        "bid": 41.85,
        "fyToken": "1011240425139434",
        "ltp": 41.75,
        "ltpch": -14.65,
        "ltpchp": -25.98,
        "oi": 338100,
        "oich": -9975,
        "oichp": -2.87,
        "option_type": "PE",
        "prev_oi": 348075,
        "strike_price": 3900,
        "symbol": "NSE:TCS24APR3900PE",
        "volume": 129325
      }
    ],
    "putOi": 3875200
  },
  "message": "",
  "s": "ok"
}
